# Fine-tune KaniTTS cho Tiếng Việt

**Datasets:** 4 subset từ VieNeu-TTS-500h (NanoCodec)
- Nam miền Nam, Nữ miền Nam, Nam miền Bắc, Nữ miền Bắc
- Mỗi subset đã được encode NanoCodec (22kHz, 0.6kbps, 12.5fps)

---


## 📦 Step 1: Install Dependencies

Cài đặt các thư viện cần thiết cho fine-tuning.


In [1]:
# Install dependencies
%uv pip install -q transformers==4.56.0 accelerate==1.10.1 datasets==3.6.0 omegaconf peft wandb
%uv pip install -q flash-attn --no-build-isolation

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from omegaconf import OmegaConf
import wandb
import os
import gc
from datetime import datetime

# Check GPU
print("=" * 70)
print("🔍 GPU INFORMATION")
print("=" * 70)
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")
print("=" * 70)

🔍 GPU INFORMATION
CUDA Available: True
GPU: NVIDIA L40S
VRAM: 50.9 GB
CUDA Version: 12.9
PyTorch Version: 2.8.0+cu129
Transformers Version: 4.56.0


## 🔐 Step 2: Login to HuggingFace & Weights & Biases


In [ ]:
import os
from huggingface_hub import login
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    login(token=HF_TOKEN)
    print("✅ Login thành công từ environment variable!")
else:
    print("⚠️ HF_TOKEN not found in environment variables")


🔐 HuggingFace Login
Lấy token từ: https://huggingface.co/settings/tokens

✅ Login thành công!


## ⚙️ Step 3: Configuration

Cấu hình model và dataset cho training.


In [ ]:
# ===== CONFIGURATION =====

# Base model
BASE_MODEL = "pnnbao-ump/kani-tts-370m-vie"

# Dataset configuration (4 subset: Nam/Nữ x Miền Nam/Bắc)
DATASET_CONFIG = {
    "max_duration_sec": 12,  # Hoặc None nếu muốn giữ nguyên độ dài
    "hf_datasets": [
        {
            "reponame": "pnnbao-ump/VieNeu-TTS-500h-nanocodec-male-south",
            "name": None,
            "split": "train",
            "text_col_name": "text",
            "nano_layer_1": "nano_layer_1",
            "nano_layer_2": "nano_layer_2",
            "nano_layer_3": "nano_layer_3",
            "nano_layer_4": "nano_layer_4",
            "encoded_len": "encoded_len",
            "speaker_id": "nam_mien_nam",
            "max_len": None,
            "categorical_filter": None,
        },
        {
            "reponame": "pnnbao-ump/VieNeu-TTS-500h-nanocodec-female-south",
            "name": None,
            "split": "train",
            "text_col_name": "text",
            "nano_layer_1": "nano_layer_1",
            "nano_layer_2": "nano_layer_2",
            "nano_layer_3": "nano_layer_3",
            "nano_layer_4": "nano_layer_4",
            "encoded_len": "encoded_len",
            "speaker_id": "nu_mien_nam",
            "max_len": None,
            "categorical_filter": None,
        },
        {
            "reponame": "pnnbao-ump/VieNeu-TTS-500h-nanocodec-male-north",
            "name": None,
            "split": "train",
            "text_col_name": "text",
            "nano_layer_1": "nano_layer_1",
            "nano_layer_2": "nano_layer_2",
            "nano_layer_3": "nano_layer_3",
            "nano_layer_4": "nano_layer_4",
            "encoded_len": "encoded_len",
            "speaker_id": "nam_mien_bac",
            "max_len": None,
            "categorical_filter": None,
        },
        {
            "reponame": "pnnbao-ump/VieNeu-TTS-500h-nanocodec-female-north",
            "name": None,
            "split": "train",
            "text_col_name": "text",
            "nano_layer_1": "nano_layer_1",
            "nano_layer_2": "nano_layer_2",
            "nano_layer_3": "nano_layer_3",
            "nano_layer_4": "nano_layer_4",
            "encoded_len": "encoded_len",
            "speaker_id": "nu_mien_bac",
            "max_len": None,
            "categorical_filter": None,
        },
    ],
}

# LoRA Configuration
LORA_CONFIG = {
    "r": 48,
    "lora_alpha": 48,
    "lora_dropout": 0.1,
    "target_modules": ['q_proj', 'k_proj', 'v_proj', 'out_proj',
                   'w1', 'w2', 'w3'],
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "use_rslora": True
}

# Training Configuration
TRAINING_CONFIG = {
    "num_train_epochs": 3,
    "per_device_train_batch_size": 8,
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-5,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "weight_decay": 0.02,
    "max_grad_norm": 1.0,
    "bf16": True,
    "optim": "adamw_torch",
    "logging_steps": 20,
    "save_steps": 500,
    "save_total_limit": 3,
    "output_dir": "./checkpoints/kani_tts_vi",
    "report_to": "wandb",
    "run_name": f"kani-tts-vi-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "gradient_checkpointing": True, 
    "gradient_checkpointing_kwargs": {"use_reentrant": False},
}

# W&B Project
WANDB_PROJECT = "kani-tts-vi-finetune"

print("✅ Configuration loaded!")
print(f"   Base Model: {BASE_MODEL}")
print(f"   Max Duration: {DATASET_CONFIG['max_duration_sec']}s")
print(f"   Số dataset: {len(DATASET_CONFIG['hf_datasets'])}")
for ds in DATASET_CONFIG["hf_datasets"]:
    print(f"     - {ds['speaker_id']}: {ds['reponame']}")
print(f"   LoRA Rank: {LORA_CONFIG['r']}")
print(f"   Batch Size: {TRAINING_CONFIG['per_device_train_batch_size']}")
print(f"   Epochs: {TRAINING_CONFIG['num_train_epochs']}")

✅ Configuration loaded!
   Base Model: nineninesix/kani-tts-370m
   Dataset: pnnbao-ump/VieNeu-TTS-140h-nanocodec
   Max Duration: Nones
   LoRA Rank: 48
   Batch Size: 8
   Epochs: 3


## 📊 Step 4: Load and Process Dataset

Load dataset từ HuggingFace và xử lý cho training.


In [6]:
import urllib.request
try:
    url = "https://raw.githubusercontent.com/nineninesix-ai/KaniTTS-Finetune-pipeline/main/dataset_processor.py"
    urllib.request.urlretrieve(url, "dataset_processor.py")
    print("✅ Đã download dataset_processor.py")
except Exception as e:
    print(f"❌ Không thể download: {e}")
    print("💡 Sẽ sử dụng code tích hợp trong notebook")


✅ Đã download dataset_processor.py


In [ ]:
import sys
import os
from torch.utils.data import Dataset
import torch
from datasets import load_dataset, concatenate_datasets
from omegaconf import OmegaConf
from transformers import AutoTokenizer
import locale
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np

locale.getpreferredencoding = lambda: "UTF-8"

# TrainDataPreProcessor class
class TrainDataPreProcessor:
    def __init__(self, tokenizer_name: str, max_dur: int, speaker_id: str = None):
        self.text_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_dur = max_dur
        self.speaker_id = speaker_id
        
        # Token configuration
        self.tokeniser_length = 64400
        self.start_of_text = 1
        self.end_of_text = 2
        self.start_of_speech = self.tokeniser_length + 1
        self.end_of_speech = self.tokeniser_length + 2
        self.start_of_human = self.tokeniser_length + 3
        self.end_of_human = self.tokeniser_length + 4
        self.start_of_ai = self.tokeniser_length + 5
        self.end_of_ai = self.tokeniser_length + 6
        self.pad_token = self.tokeniser_length + 7
        self.audio_tokens_start = self.tokeniser_length + 10
        self.codebook_size = 4032
    
    def add_codes(self, example):
        snac_layers = ['nano_layer_1', 'nano_layer_2', 'nano_layer_3', 'nano_layer_4']
        codes = [example[i] for i in snac_layers]
        codes = np.array(codes).T
        all_codes = codes + np.array([self.codebook_size * i for i in range(4)])
        
        # remove duplicates
        mask = np.any(all_codes[1:] != all_codes[:-1], axis=1)
        keep = np.insert(mask, 0, True)
        all_codes = all_codes[keep]
        
        # flatten to sequence
        all_codes = all_codes + self.audio_tokens_start
        example["codes_list"] = all_codes.flatten().tolist()
        return example
    
    def create_input_ids(self, example):
        if self.speaker_id is not None:
            text_prompt = f"{self.speaker_id.lower()}: {example['text']}"
        else:
            text_prompt = example["text"]
        
        text_ids = self.text_tokenizer.encode(text_prompt, add_special_tokens=True)
        text_ids.append(self.end_of_text)
        
        example["text_tokens"] = text_ids
        input_ids = (
            [self.start_of_human]
            + example["text_tokens"]
            + [self.end_of_human]
            + [self.start_of_ai]
            + [self.start_of_speech]
            + example["codes_list"]
            + [self.end_of_speech]
            + [self.end_of_ai]
        )
        example["input_ids"] = input_ids
        example["labels"] = input_ids
        example["attention_mask"] = [1] * len(input_ids)
        return example
    
    def __call__(self, dataset: Dataset) -> Dataset:
        print(f'🔄 Processing dataset with {len(dataset)} samples...')
        
        if self.max_dur:
            fps = 12.5
            dataset_len = len(dataset)
            dataset = dataset.filter(lambda i: i['encoded_len']/fps <= self.max_dur)
            filtred_len = len(dataset)
            print(f'📊 Filtered by duration: {filtred_len} rows from {dataset_len}')
        
        dataset = dataset.map(self.add_codes, remove_columns=['nano_layer_1', 'nano_layer_2', 'nano_layer_3', 'nano_layer_4'], desc='Add Audio Codes')
        dataset = dataset.filter(lambda x: x["codes_list"] is not None, desc='Check codes list')
        dataset = dataset.filter(lambda x: len(x["codes_list"]) > 0, desc='Check Codes list length')
        dataset = dataset.map(self.create_input_ids, remove_columns=["text", "codes_list"], desc='Create input ids')
        
        columns_to_keep = ["input_ids", "labels", "attention_mask"]
        columns_to_remove = [col for col in dataset.column_names if col not in columns_to_keep]
        dataset = dataset.remove_columns(columns_to_remove)
        
        print(f'✅ Processing completed: {len(dataset)} samples')
        return dataset

def process_shard(shard_idx, shard_data, tokenizer_name, max_dur, speaker_id):
    print(f'🚀 WORKER {shard_idx}: Starting processing...')
    processor = TrainDataPreProcessor(tokenizer_name, max_dur, speaker_id)
    processed_shard = processor(shard_data)
    print(f'✅ WORKER {shard_idx}: Completed processing')
    return processed_shard

class ItemDataset:
    def __init__(self, item_cfg: OmegaConf, tokenizer_name: str, max_dur: int, n_shards: int = None):
        print(f'📦 Loading dataset "{item_cfg.reponame}"...')
        self.item_cfg = item_cfg
        self.tokenizer_name = tokenizer_name
        self.max_dur = max_dur
        self.speaker_id = item_cfg.get('speaker_id')
        self.max_len = item_cfg.get('max_len')
        
        if n_shards is None:
            self.n_shards = min(mp.cpu_count(), 8)
        else:
            self.n_shards = n_shards
        
        self.dataset = load_dataset(
            item_cfg.reponame,
            item_cfg.name,
            split=item_cfg.split,
            num_proc=10
        )
        
        print(f'📊 Loaded {len(self.dataset)} samples')
        
        if item_cfg.get('categorical_filter'):
            print(f'🔧 Filtering by {item_cfg.categorical_filter.column_name} = {item_cfg.categorical_filter.value}')
            self.dataset = self.dataset.filter(
                lambda x: x[item_cfg.categorical_filter.column_name] == item_cfg.categorical_filter.value
            )
            print(f'✅ Filtered to {len(self.dataset)} samples')
        
        print(f'🔄 Renaming columns...')
        rename_dict = {
            item_cfg.text_col_name: 'text',
            item_cfg.nano_layer_1: 'nano_layer_1',
            item_cfg.nano_layer_2: 'nano_layer_2',
            item_cfg.nano_layer_3: 'nano_layer_3',
            item_cfg.nano_layer_4: 'nano_layer_4',
            item_cfg.encoded_len: 'encoded_len',
        }
        self.dataset = self.dataset.rename_columns(rename_dict)
        print(f'✅ Column renaming completed')
    
    def __call__(self):
        print(f'🔄 Starting parallel processing with {self.n_shards} shards...')
        
        shards = []
        for i in range(self.n_shards):
            shard = self.dataset.shard(num_shards=self.n_shards, index=i)
            shards.append((shard, i))
            print(f'📦 SHARD {i}: Created with {len(shard)} samples')
        
        processed_shards = []
        with ProcessPoolExecutor(max_workers=self.n_shards) as executor:
            future_to_shard = {
                executor.submit(process_shard, shard_idx, shard, self.tokenizer_name, self.max_dur, self.speaker_id): shard_idx
                for shard, shard_idx in shards
            }
            
            for future in as_completed(future_to_shard):
                shard_idx = future_to_shard[future]
                try:
                    processed_shard = future.result()
                    processed_shards.append((shard_idx, processed_shard))
                    print(f'✅ COMPLETED: Shard {shard_idx} processing finished')
                except Exception as exc:
                    print(f'❌ ERROR: Shard {shard_idx} generated an exception: {exc}')
                    raise exc
        
        processed_shards.sort(key=lambda x: x[0])
        final_shards = [shard for _, shard in processed_shards]
        
        print(f'🔗 Concatenating {len(final_shards)} processed shards...')
        final_dataset = concatenate_datasets(final_shards)
        
        if self.max_len is not None:
            final_dataset = final_dataset.shuffle(seed=42).select(range(self.max_len))
        
        print(f'✅ Dataset processing completed! Final size: {len(final_dataset)} samples')
        return final_dataset

# Create dataset config
print("=" * 70)
print("📥 LOADING AND PROCESSING DATASETS")
print("=" * 70)
print(f"Max duration (seconds): {DATASET_CONFIG['max_duration_sec']}")
print(f"Tổng số dataset sẽ load: {len(DATASET_CONFIG['hf_datasets'])}")
print("=" * 70)

dataset_summaries = []
processed_datasets = []

for ds_idx, ds_cfg in enumerate(DATASET_CONFIG["hf_datasets"], start=1):
    item_cfg_dict = {
        "reponame": ds_cfg["reponame"],
        "name": ds_cfg.get("name"),
        "split": ds_cfg.get("split", "train"),
        "text_col_name": ds_cfg["text_col_name"],
        "nano_layer_1": ds_cfg["nano_layer_1"],
        "nano_layer_2": ds_cfg["nano_layer_2"],
        "nano_layer_3": ds_cfg["nano_layer_3"],
        "nano_layer_4": ds_cfg["nano_layer_4"],
        "encoded_len": ds_cfg["encoded_len"],
        "speaker_id": ds_cfg.get("speaker_id"),
        "max_len": ds_cfg.get("max_len"),
        "categorical_filter": ds_cfg.get("categorical_filter"),
    }

    item_cfg = OmegaConf.create(item_cfg_dict)

    print("\n" + "-" * 70)
    print(f"🎯 Dataset {ds_idx}: {ds_cfg['reponame']}")
    print(f"    Speaker ID prompt : {ds_cfg.get('speaker_id')}")
    if ds_cfg.get("categorical_filter"):
        filt = ds_cfg["categorical_filter"]
        print(f"    Filter: {filt['column_name']} = {filt['value']}")
    if ds_cfg.get("max_len"):
        print(f"    Max samples: {ds_cfg['max_len']}")

    item_dataset = ItemDataset(
        item_cfg=item_cfg,
        tokenizer_name=BASE_MODEL,
        max_dur=DATASET_CONFIG["max_duration_sec"],
        n_shards=8,
    )

    subset_dataset = item_dataset()
    subset_dataset = subset_dataset.shuffle(seed=42)

    processed_datasets.append(subset_dataset)
    dataset_summaries.append(
        {
            "speaker_id": ds_cfg.get("speaker_id"),
            "reponame": ds_cfg["reponame"],
            "num_samples": len(subset_dataset),
        }
    )

if not processed_datasets:
    raise RuntimeError("Không load được dataset nào. Kiểm tra cấu hình DATASET_CONFIG.")

full_dataset = concatenate_datasets(processed_datasets)
full_dataset = full_dataset.shuffle(seed=42)

# ✅ THÊM: Split train/validation (10% cho validation)
VALIDATION_SPLIT = 0.1  # 10% cho validation
total_samples = len(full_dataset)
val_samples = int(total_samples * VALIDATION_SPLIT)

train_dataset = full_dataset.select(range(total_samples - val_samples))
val_dataset = full_dataset.select(range(total_samples - val_samples, total_samples))

print("\n" + "=" * 70)
print("✅ DATASET READY!")
print("=" * 70)
print(f"Total samples: {total_samples:,}")
print(f"Train samples: {len(train_dataset):,} ({100*(1-VALIDATION_SPLIT):.1f}%)")
print(f"Validation samples: {len(val_dataset):,} ({100*VALIDATION_SPLIT:.1f}%)")
print(f"Features: {train_dataset.column_names}")
print("-" * 70)
for summary in dataset_summaries:
    print(
        f"   • {summary['speaker_id']}: {summary['num_samples']:,} samples ({summary['reponame']})"
    )
print("=" * 70)
print("\n💡 Validation set sẽ được dùng để:")
print("   - Phát hiện overfitting (quan trọng với LoRA rank cao)")
print("   - Chọn best model tự động")
print("   - Monitor training progress")
print("=" * 70)


📥 LOADING AND PROCESSING DATASET
Dataset: pnnbao-ump/VieNeu-TTS-140h-nanocodec
Max samples: None
Speaker ID: None
Max duration: Nones
📦 Loading dataset "pnnbao-ump/VieNeu-TTS-140h-nanocodec"...


README.md:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/58.3M [00:00<?, ?B/s]

Setting num_proc from 10 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split:   0%|          | 0/74858 [00:00<?, ? examples/s]

...

Create input ids:   0%|          | 0/9357 [00:00<?, ? examples/s]

Create input ids:   0%|          | 0/9357 [00:00<?, ? examples/s]

Create input ids:   0%|          | 0/9358 [00:00<?, ? examples/s]

Create input ids:   0%|          | 0/9358 [00:00<?, ? examples/s]

✅ Processing completed: 9357 samples
✅ WORKER 4: Completed processing
✅ Processing completed: 9357 samples
✅ WORKER 6: Completed processing
✅ Processing completed: 9357 samples
✅ WORKER 2: Completed processing
✅ Processing completed: 9357 samples✅ Processing completed: 9357 samples

✅ WORKER 7: Completed processing✅ WORKER 3: Completed processing

✅ Processing completed: 9357 samples
✅ WORKER 5: Completed processing✅ Processing completed: 9358 samples✅ Processing completed: 9358 samples


✅ WORKER 1: Completed processing✅ WORKER 0: Completed processing

✅ COMPLETED: Shard 4 processing finished
✅ COMPLETED: Shard 6 processing finished
✅ COMPLETED: Shard 2 processing finished
✅ COMPLETED: Shard 7 processing finished
✅ COMPLETED: Shard 3 processing finished
✅ COMPLETED: Shard 5 processing finished
✅ COMPLETED: Shard 1 processing finished
✅ COMPLETED: Shard 0 processing finished
🔗 Concatenating 8 processed shards...
✅ Dataset processing completed! Final size: 74858 samples

✅ DATASET READY

## 🧠 Step 5: Load Model and Tokenizer

Load base model và tokenizer, sau đó apply LoRA.


In [8]:
print("📚 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
print(f"   Vocab size: {tokenizer.vocab_size}")

print("\n🧠 Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # H100 hỗ trợ tốt bf16
    attn_implementation="flash_attention_2"  # Tận dụng Flash Attention trên H100
)

print(f"\n✅ Model loaded!")
print(f"   Device: {next(model.parameters()).device}")
print(f"   Dtype: {next(model.parameters()).dtype}")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Check GPU memory
if torch.cuda.is_available():
    print(f"\n💾 GPU Memory:")
    print(f"   Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   Reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    print(f"   Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved(0)) / 1e9:.2f} GB")


📚 Loading tokenizer...
   Vocab size: 64400

🧠 Loading model...


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]


✅ Model loaded!
   Device: cuda:0
   Dtype: torch.bfloat16
   Parameters: 369,847,040

💾 GPU Memory:
   Allocated: 0.75 GB
   Reserved: 0.79 GB
   Free: 50.07 GB


In [9]:
print("🔧 Applying LoRA...")
lora_config = LoraConfig(**LORA_CONFIG)
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n✅ LoRA applied!")
print(f"   Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"   Total parameters: {total_params:,}")

# Check GPU memory after LoRA
if torch.cuda.is_available():
    print(f"\n💾 GPU Memory (after LoRA):")
    print(f"   Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   Reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")


🔧 Applying LoRA...

✅ LoRA applied!
   Trainable parameters: 16,023,552 (4.15%)
   Total parameters: 385,870,592

💾 GPU Memory (after LoRA):
   Allocated: 0.81 GB
   Reserved: 0.86 GB


## 🚀 Step 6: Initialize Training

Setup trainer và bắt đầu training.


In [ ]:
# Initialize W&B
wandb.init(
    project=WANDB_PROJECT,
    name=TRAINING_CONFIG["run_name"],
    config={
        **TRAINING_CONFIG,
        **LORA_CONFIG,
        "base_model": BASE_MODEL,
        "max_duration_sec": DATASET_CONFIG["max_duration_sec"],
        "dataset_repos": [ds["reponame"] for ds in DATASET_CONFIG["hf_datasets"]],
        "speaker_ids": [ds.get("speaker_id") for ds in DATASET_CONFIG["hf_datasets"]],
        "max_samples_per_repo": [ds.get("max_len") for ds in DATASET_CONFIG["hf_datasets"]],
    }
)

# Ensure tokenizer has pad_token
pad_token_id = 64407
if tokenizer.pad_token is None:
    tokenizer.pad_token_id = pad_token_id
    print(f"✅ Set pad_token_id to {pad_token_id}")

# Custom Data Collator (GIỐNG CŨ, KHÔNG ĐỔI)
from transformers import DataCollatorForLanguageModeling
import torch

class CustomDataCollator(DataCollatorForLanguageModeling):
    def __call__(self, features):
        labels = [f.pop("labels") for f in features] 
        
        batch = self.tokenizer.pad(
            features,
            padding=True,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        max_length = batch["input_ids"].shape[1]
        padded_labels = []
        for label in labels:
            padded_label = label + [-100] * (max_length - len(label))
            padded_labels.append(padded_label)
        
        batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)
        
        return batch

data_collator = CustomDataCollator(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8,
)

# ✅ TÍNH TOÁN EVAL_STEPS VÀ SAVE_STEPS
steps_per_epoch = len(train_dataset) // (
    TRAINING_CONFIG['per_device_train_batch_size'] * 
    TRAINING_CONFIG['gradient_accumulation_steps']
)
eval_steps = max(250, steps_per_epoch // 4)  # ✅ Eval 4 lần mỗi epoch

print("=" * 70)
print("📊 TRAINING CONFIGURATION")
print("=" * 70)
print(f"Training samples: {len(train_dataset):,}")
print(f"Validation samples: {len(val_dataset):,}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Eval steps: {eval_steps} (will eval ~{steps_per_epoch // eval_steps} times per epoch)")
print(f"Total training steps: {steps_per_epoch * TRAINING_CONFIG['num_train_epochs']}")
print("=" * 70)

# ✅ TRAINING ARGUMENTS VỚI FULL VALIDATION SUPPORT
from transformers import TrainingArguments, EarlyStoppingCallback

training_config_dict = TRAINING_CONFIG.copy()
training_config_dict.pop('save_steps', None)
training_config_dict.pop('save_total_limit', None)

training_args = TrainingArguments(
    **training_config_dict,
    overwrite_output_dir=True,
    remove_unused_columns=True,
    
    # ✅ VALIDATION & EVALUATION
    eval_strategy="steps",
    eval_steps=eval_steps,
    do_eval=True,  # ✅ THÊM: Explicitly enable evaluation
    
    # ✅ CHECKPOINTING
    save_strategy="steps",
    save_steps=eval_steps,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
    
    # ✅ MONITORING
    logging_first_step=True,  # ✅ Log ngay step đầu
    eval_on_start=True,  # ✅ Eval trước khi train để có baseline
)

print("\n🗂️  Creating trainer...")

# ✅ CHECK MODEL STATE
from peft import PeftModel
if not isinstance(model, PeftModel):
    print("⚠️ Model chưa được apply LoRA! Đang apply lại...")
    lora_config = LoraConfig(**LORA_CONFIG)
    model = get_peft_model(model, lora_config)
    print("✅ LoRA đã được apply lại!")

model.train()
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Trainable parameters: {trainable_params:,}")
if trainable_params == 0:
    raise RuntimeError("❌ Model không có trainable parameters! Kiểm tra lại LoRA config.")

# ✅ EARLY STOPPING CALLBACK
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Stop if no improvement for 3 evals
    early_stopping_threshold=0.001  # Minimum improvement threshold
)

# ✅ CREATE TRAINER WITH EARLY STOPPING
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping_callback],  # ✅ Add early stopping
)

print(f"\n✅ Trainer ready!")
print(f"   Total training steps: {trainer.state.max_steps if hasattr(trainer.state, 'max_steps') else steps_per_epoch * TRAINING_CONFIG['num_train_epochs']}")
print(f"   Eval strategy: {training_args.eval_strategy}")
print(f"   Eval steps: {eval_steps}")
print(f"   Early stopping patience: 3 evaluations")
print(f"   Gradient checkpointing: {TRAINING_CONFIG.get('gradient_checkpointing', False)}")
print("=" * 70)

wandb: Tracking run with wandb version 0.22.3
wandb: Run data is saved locally in /root/wandb/run-20251107_020414-69fqyi6w
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run kani-tts-vi-20251107-020347
wandb: ⭐️ View project at https://wandb.ai/scream3ktr6-ewyss/vietnamese-tts-finetune
wandb: 🚀 View run at https://wandb.ai/scream3ktr6-ewyss/vietnamese-tts-finetune/runs/69fqyi6w
/tmp/ipykernel_160/72077341.py:124: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


📊 TRAINING CONFIGURATION
Training samples: 67,373
Validation samples: 7,485
Steps per epoch: 2105
Eval steps: 526 (will eval ~4 times per epoch)
Total training steps: 6315

🗂️  Creating trainer...
   Trainable parameters: 16,023,552

✅ Trainer ready!
   Total training steps: 0
   Eval strategy: IntervalStrategy.STEPS
   Eval steps: 526
   Early stopping patience: 3 evaluations
   Gradient checkpointing: True


## 🎯 Step 7: Start Training


In [11]:
import time

print("=" * 70)
print("🚀 STARTING TRAINING")
print("=" * 70)
print(f"Dataset size: {len(train_dataset):,} samples")
print(f"Validation size: {len(val_dataset):,} samples")
print(f"Batch size: {TRAINING_CONFIG['per_device_train_batch_size']}")
print(f"Gradient accumulation: {TRAINING_CONFIG['gradient_accumulation_steps']}")
print(f"Effective batch size: {TRAINING_CONFIG['per_device_train_batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']}")
print(f"Epochs: {TRAINING_CONFIG['num_train_epochs']}")
print(f"Eval steps: {eval_steps} (validation sẽ được chạy mỗi {eval_steps} steps)")
print("=" * 70)
print("\n💡 Bạn sẽ thấy validation loss trong:")
print("   - Console output (mỗi eval_steps)")
print("   - W&B dashboard (train_loss vs eval_loss)")
print("=" * 70)

start_time = time.time()

# Start training
trainer.train()

end_time = time.time()
training_time = end_time - start_time

print("\n" + "=" * 70)
print("✅ TRAINING COMPLETED!")
print("=" * 70)
print(f"Total training time: {training_time / 3600:.2f} hours ({training_time / 60:.1f} minutes)")
print(f"Average time per epoch: {training_time / TRAINING_CONFIG['num_train_epochs'] / 60:.1f} minutes")
print("=" * 70)
print("\n📊 Validation results:")
print(f"   Best eval_loss: {trainer.state.best_metric if hasattr(trainer.state, 'best_metric') else 'N/A'}")
print(f"   Best model checkpoint: {trainer.state.best_model_checkpoint if hasattr(trainer.state, 'best_model_checkpoint') else 'N/A'}")
print("=" * 70)

wandb.finish()


🚀 STARTING TRAINING
Dataset size: 67,373 samples
Validation size: 7,485 samples
Batch size: 8
Gradient accumulation: 4
Effective batch size: 32
Epochs: 3
Eval steps: 526 (validation sẽ được chạy mỗi 526 steps)

💡 Bạn sẽ thấy validation loss trong:
   - Console output (mỗi eval_steps)
   - W&B dashboard (train_loss vs eval_loss)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
0,No log,5.726142
526,4.949700,4.942051
1052,4.652100,4.662650
1578,4.595200,4.588301
2104,4.541400,4.549474
2630,4.502600,4.525425
3156,4.489000,4.507982
3682,4.506300,4.493707
4208,4.480600,4.483937
4734,4.462400,4.478301


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
wandb: updating run metadata



✅ TRAINING COMPLETED!
Total training time: 1.98 hours (118.8 minutes)
Average time per epoch: 39.6 minutes

📊 Validation results:
   Best eval_loss: 4.473038673400879
   Best model checkpoint: ./checkpoints/kani_tts_vi/checkpoint-6312


wandb: uploading history steps 329-329, summary
wandb: 
wandb: Run history:
wandb:               eval/loss █▄▂▂▁▁▁▁▁▁▁▁▁
wandb:            eval/runtime ▂▄▁▄▁▂█▂▂▂▄▄▄
wandb: eval/samples_per_second ▇▅█▅█▇▁▇▇▇▅▅▅
wandb:   eval/steps_per_second ▇▅█▅█▇▁▇▇▇▅▅▅
wandb:             train/epoch ▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:       train/global_step ▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇███
wandb:         train/grad_norm ▃▃▁▂▄▅▅▅█▅▅▆▆▆▆▆▆▇▆▆▆▇▇▇▆▇▇▇▆▇▆▆▆▆▇▆▆▇▆▆
wandb:     train/learning_rate ▃▇█████████▇▇▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
wandb:              train/loss ██▆▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:               eval/loss 4.47304
wandb:            eval/runtime 72.0898
wandb: eval/samples_per_second 103.829
wandb:   eval/steps_per_second 12.984
wandb:              total_flos 2.5952487538634342e+17
wandb:             train/epoch 3
wandb:       train/global_step 6318
wandb:         train/grad_norm 0.96209
wandb:     train/learning_rate 0.0
wand

## 💾 Step 8: Save Model

Merge LoRA weights và lưu model.


In [12]:
print("🔄 Merging LoRA weights...")
merged_model = model.merge_and_unload()

output_path = TRAINING_CONFIG["output_dir"]
os.makedirs(output_path, exist_ok=True)

print(f"💾 Saving model to {output_path}...")
merged_model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print(f"\n✅ Model saved successfully!")
print(f"   Path: {output_path}")
print(f"\n📤 To upload to HuggingFace Hub:")
print(f"   huggingface-cli upload <your-username>/vietnamese-tts-model {output_path} --private")


🔄 Merging LoRA weights...
💾 Saving model to ./checkpoints/kani_tts_vi...

✅ Model saved successfully!
   Path: ./checkpoints/kani_tts_vi

📤 To upload to HuggingFace Hub:
   huggingface-cli upload <your-username>/vietnamese-tts-model ./checkpoints/kani_tts_vi --private


In [16]:
!huggingface-cli upload pnnbao-ump/kani-tts-370m-vi "./checkpoints/kani_tts_vi" --private

⚠️  Warning: 'huggingface-cli upload' is deprecated. Use 'hf upload' instead.
It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`hf upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.
Start hashing 43 files.
Finished hashing 43 files.
Processing Files (0 / 0)                : |        |  0.00B /  0.00B            
New Data Upload                         : |        |  0.00B /  0.00B            

  ...ts_vi/checkpoint-5786/rng_state.pth:  77%|███ | 11.3kB / 14.6kB            


  ...ts_vi/checkpoint-6318/rng_state.pth:  77%|███ | 11.3kB / 14.6kB            



  ...ts_vi/checkpoint-6312/rng_state.pth:  77%|███ | 11.3kB / 14.6kB            




  ...tts_vi/checkpoint-5786/optimizer.pt:   0%|    |  568kB /  128MB   

## 📊 Step 9: Training Summary

Xem lại kết quả training.


In [ ]:
print("=" * 70)
print("📊 TRAINING SUMMARY")
print("=" * 70)
print(f"Base Model: {BASE_MODEL}")
print("Datasets:")
for summary in dataset_summaries:
    print(f"   - {summary['speaker_id']}: {summary['reponame']} ({summary['num_samples']:,} samples)")
print(f"Training Samples: {len(train_dataset):,}")
print(f"LoRA Rank: {LORA_CONFIG['r']}")
print(f"Batch Size: {TRAINING_CONFIG['per_device_train_batch_size']}")
print(f"Epochs: {TRAINING_CONFIG['num_train_epochs']}")
print(f"Learning Rate: {TRAINING_CONFIG['learning_rate']}")
print(f"Training Time: {training_time / 3600:.2f} hours")
print(f"Model Saved: {output_path}")
print("=" * 70)

# Check final GPU memory
if torch.cuda.is_available():
    print(f"\n💾 Final GPU Memory:")
    print(f"   Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   Reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    print(f"   Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved(0)) / 1e9:.2f} GB")

print("\n🎉 Fine-tuning completed successfully!")
print("\n💡 Next steps:")
print("   1. Test the model with inference script")
print("   2. Upload to HuggingFace Hub if needed")
print("   3. Evaluate audio quality")


📊 TRAINING SUMMARY
Base Model: nineninesix/kani-tts-370m
Dataset: pnnbao-ump/VieNeu-TTS-140h-nanocodec
Training Samples: 67,373
LoRA Rank: 48
Batch Size: 8
Epochs: 3
Learning Rate: 2e-05
Training Time: 1.98 hours
Model Saved: ./checkpoints/kani_tts_vi

💾 Final GPU Memory:
   Allocated: 0.96 GB
   Reserved: 24.36 GB
   Free: 26.51 GB

🎉 Fine-tuning completed successfully!

💡 Next steps:
   1. Test the model with inference script
   2. Upload to HuggingFace Hub if needed
   3. Evaluate audio quality
